In [ ]:
import sys
from pathlib import Path

# Add the src directory to sys.path
root_dir = Path().absolute().parent.parent
# Add project root to the path of Python
sys.path.append(str(root_dir))

from src import config  # noqa: E402

settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import datetime
import json
import warnings

import hopsworks
import pandas as pd

from src.utils import util

warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [5]:
project = hopsworks.login()
fs = project.get_feature_store()
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country = location["country"]
city = location["city"]
street = location["street"]
aqicn_url = location["aqicn_url"]
latitude = location["latitude"]
longitude = location["longitude"]

today = datetime.date.today()

location_str

2025-05-18 10:35:12,005 INFO: Initializing external client
2025-05-18 10:35:12,005 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-18 10:35:13,867 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1224891


'{"country": "colombia", "city": "medellin", "street": "el-poblado", "aqicn_url": "https://api.waqi.info/feed/@12635", "latitude": 6.27, "longitude": -75.6}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [6]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name="air_quality",
    version=1,
)
weather_fg = fs.get_feature_group(
    name="weather",
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [7]:
aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,38.0,colombia,medellin,el-poblado,2025-05-18,https://api.waqi.info/feed/@12635


In [8]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [9]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index("date")

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time("11:59", "12:01")
daily_df = daily_df.reset_index()
daily_df["date"] = pd.to_datetime(daily_df["date"]).dt.date
daily_df["date"] = pd.to_datetime(daily_df["date"])
daily_df["city"] = city
daily_df

Coordinates 6.25°N -75.75°E
Elevation 1570.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-05-18,16.400000,0.4,2.160000,90.000000,medellin
1,2025-05-19,16.850000,0.0,1.800000,143.130020,medellin
2,2025-05-20,17.250000,0.0,2.545584,81.869987,medellin
3,2025-05-21,17.850000,0.1,2.160000,90.000000,medellin
4,2025-05-22,17.500000,0.0,5.692100,108.435043,medellin
5,2025-05-23,18.049999,0.3,3.706427,150.945496,medellin
6,2025-05-24,17.200001,0.2,4.104631,105.255173,medellin
7,2025-05-25,17.400000,0.0,4.735060,81.253922,medellin
8,2025-05-26,17.100000,0.0,5.411986,93.814003,medellin
9,2025-05-27,17.500000,0.6,1.835647,168.690109,medellin


In [10]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 452.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [11]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-05-18 10:35:43,014 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1224891/fs/1211503/fg/1458858


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1224891/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 709663
         }
       },
       "result": {
         "observed_value": 37.999996185302734,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-05-18T03:35:43.000013Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "succe

In [10]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-01-09 15:11:57,920 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/398/fs/335/fg/1394843


Uploading Dataframe: 100.00% |███████████████████████████████████████████████████████████████████████████████████████████| Rows 10/10 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/398/jobs/named/weather_1_offline_fg_materialization/executions


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 694652
         }
       },
       "result": {
         "observed_value": 5.091168403625488,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-09T02:11:57.000920Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type":

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
